<a href="https://colab.research.google.com/github/ayushjain1144/NER/blob/master/NER_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import sys

# Loading the features and vocabulary

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s content/gdrive/My\ Drive/NER /ner_dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ..
%cd /ner_dir


/
/content/gdrive/My Drive/NER


In [4]:
!ls

activations.py		  loss.py		README.md
Conll.ipynb		  NER_NN.ipynb		test_features.npy
dataset			  NER_NN_network.ipynb	train_features.npy
initial_experiment.ipynb  nn.py			val_features.npy
initialization.py	  __pycache__		vocab.npy


In [5]:
import activations
import loss
import initialization as init_layer

In [6]:
vocab = np.load('vocab.npy')
train_features = np.load('train_features.npy')
test_features = np.load('test_features.npy')
val_features = np.load('val_features.npy')

In [7]:
train_features.shape

(204566, 900)

In [8]:
C = 1
word_vector_dim = 300
num_tags = 10

# Neural Netwwork Architecture

In [9]:
nn_architecture = [
  {"layer_size": 900, "activation": "none"},
  {"layer_size": 300, "activation": "relu"},
  {"layer_size": 100, "activation": "relu"},
  {"layer_size":10, "activation": "softmax"}
]

# Initialize Parameters

In [10]:
def initialize_parameters(nn_architecture, initialization = "range_initialization", seed=5):

  parameters = {}
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):

    if initialization == "range_initialization":
      parameters['W' + str(i)] = init_layer.range_initializtion(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)
    else:
      parameters['W' + str(i)] = init_layer.random_initializtion(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)

    parameters['b' + str(i)] = np.zeros((nn_architecture[i]["layer_size"], 1))

  return parameters

In [ ]:
initialize_parameters(nn_architecture, initialization="range_initialization")

# Forward Propogation

In [12]:
# Z = W * X + b
# Here A is output of previous layer

def linear_forward(A_prev, W, b):
  return np.dot(W, A_prev) + b

# apply activation h:  A = h(X) 
def apply_activation(A, activation, alpha=0.01):

  if activation == "sigmoid":
    return activations.sigmoid(A)
  elif activation == "tanh":
    return activations.tanh(A)
  elif activation == "relu":
    return activations.relu(A)
  elif activation == "leaky_relu":
    return activations.leaky_relu(A, alpha)
  else:
    print(f"ERROR: {activation} activation not supported")
    sys.exit(1)
  
# driver forward propogation
def model_forward(X, parameters, nn_architecture, alpha=0.01):

  forward_cache = {}
  A = X
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):
    A_prev = A
    W = parameters["W" + str(i)]
    b = parametes["b" + str(i)]

    Z = linear_forward(A_prev, W, b)
    A = apply_activation(Z, activation, alpha)

    forward_cache['Z' + str(i)] = Z
    forward_cache['A' + str(i - 1)] = A

  return A, forward_cache  

# Loss